## Fundamentação Comportamental e Feature Engineering

#### Inteligência Preditiva para Intervenção de Crédito Responsável

Justificativa da necessidade do modelo de ML e Dicionário de Features baseadas no risco brasileiro, **validado por resultados Out-of-Time (OOT)**.

---

### 1. Introdução: O Ciclo Vicioso e a Tese da Intervenção Proativa

#### 1.1. A Dor de Mercado e a Solução Otimizada:

**Problema:** O cliente entra no ciclo vicioso do endividamento ao usar produtos com taxas de juros elevadas (Rotativo do Cartão e Fatura Parcelada).

**Tese de Intervenção (Solução Validada):** Nossa tese é intervir no **Estresse Agudo** (dias antes do vencimento ou nos primeiros 30 dias de rotativo) com uma **oferta de crédito pessoal** com taxas mais baixas. O modelo de ML identifica esse risco com alta precisão, permitindo à FinTech converter um risco de perda (inadimplência) em um **ativo de portfólio sustentável** (empréstimo pessoal).

**Impacto Estratégico do Trade-Off:** O modelo foi otimizado para um alto **Recall (89%)**, resultando em muitos **Falsos Positivos (68%)**. Essa alta taxa de Falsos Positivos não é um custo, mas uma oportunidade: o alerta é usado para oferecer um benefício (crédito mais barato), transformando clientes que não estavam em risco real em **clientes fidelizados e bem atendidos**.

---

### 2. Panorama do Endividamento no Brasil (Dados Macro - BACEN)

**Objetivo:** Usar dados oficiais para demonstrar que o problema é sistêmico, justificando a urgência da intervenção preditiva.

| Variável BACEN | Fonte (API) | Análise e Insight |
| :--- | :--- | :--- |
| Endividamento das famílias (em relação à RNDBF) | bcdata.sgs.29038 | Mostrar que o comprometimento de renda está em patamar elevado, justificando a urgência de um PCR Ideal (Percentual de Comprometimento de Renda) que seja sustentável e adaptado à volatilidade brasileira. |

![Endividamento da Família Brasileira](https://github.com/DeboraKlein/FinSight/blob/main/data/assets/endividamento_familia.png?raw=true)

| Variável BACEN | Fonte (API) | Análise e Insight |
| :--- | :--- | :--- |
| Inadimplência da carteira de crédito - Pessoas físicas - Total | bcdata.sgs.21084 | Plotar a série histórica da inadimplência total. O modelo de ML é a ferramenta para desviar dessa média e manter a inadimplência da Fintech abaixo do mercado. |

![Inadimplência da Carteira](https://github.com/DeboraKlein/FinSight/blob/main/data/assets/inadimplencia_carteira.png?raw=true)

---

### 3. O Foco no Estresse Agudo: Cartão de Crédito e Sinais Dinâmicos

**Objetivo:** Isolar as linhas de crédito que são o alvo direto da intervenção e as features que capturam a mudança de comportamento.

| Variável BACEN | Fonte (API) | Conclusão para o Projeto |
| :--- | :--- | :--- |
| Concessões de crédito - Cheque especial | bcdata.sgs.20665 | O Cheque Especial e o Rotativo são os principais gatilhos de estresse. A Feature Engineering foca em **médias móveis e taxas de crescimento** para capturar o uso recorrente e a aceleração do endividamento. |
| Concessões de crédito - Cartão de crédito rotativo | bcdata.sgs.20679 | O uso do Rotativo é um forte preditor. Nossa intervenção permite à Fintech oferecer a **liquidação do Rotativo com Empréstimo Pessoal** antes da Resolução BACEN 4.549 obrigar o parcelamento (uma solução pior para o cliente). |

![Evolução do Cheque Especial e Cartão de Crédito](https://github.com/DeboraKlein/FinSight/blob/main/data/assets/evolucao_ch_especial_cartao_credito.png?raw=true)

---

### 4. Fatores de Risco Comportamental (Validados pelo Modelo Final)

**Objetivo:** Justificar a inclusão de features que foram comprovadamente os maiores preditores do risco no modelo LightGBM.

| Risco Comportamental (Teoria) | Implicação para a Feature Engineering | Validação (Importância do Modelo) |
| :--- | :--- | :--- |
| **Estresse de Liquidez Imediata** | O modelo precisa de proxies para o esgotamento do limite do cartão. | **Feature #1 (Mais Importante):** `num__utilizacao_limite_media_3m` |
| **Risco Sistêmico/Deterioração** | Capturar o aumento acelerado da dívida total do cliente em todo o mercado financeiro (SCR). | **Feature #2 (Altamente Preditiva):** `num__scr_crescim_divida_3m` |
| **Estresse de Consumo** | Captar a velocidade do aumento do gasto que precede a incapacidade de pagamento. | **Feature Top 5:** `num__gasto_crescim_3m` |
| **Risco Sócio-demográfico Base** | Usado para contextualização e *fine-tuning* da precificação. | `faixa_etaria_risco` e `regiao_inadimplencia_rate` (Baixa importância, contextual) |

---

### 5. Feature Set Final: Dicionário de Variáveis e Validação Preditiva

O conjunto de features foi desenhado em três pilares para garantir a máxima predição do Estresse Agudo, conforme a **Validação Out-of-Time (OOT)** do modelo LightGBM:

#### 5.1. Pilares e Features Otimizadas

| Categoria | Feature | Justificativa e Importância Comprovada |
| :--- | :--- | :--- |
| **Pilar 1: Liquidez e Estresse Agudo** | `num__utilizacao_limite_media_3m` | **#1 no ranking de importância.** Captura a dependência do cliente em relação ao limite do cartão. | 
| **Pilar 1: Liquidez e Estresse Agudo** | `num__gasto_crescim_3m` | **Top 5 no ranking.** Capta a aceleração do gasto que é um sintoma do estresse de caixa. |
| **Pilar 2: Risco Sistêmico (SCR)** | `num__scr_crescim_divida_3m` | **#2 no ranking de importância.** Prova o aumento da dívida do cliente em todo o mercado financeiro. |
| **Pilar 2: Risco Sistêmico (SCR)** | `num__scr_divida_inicial` | **Top 5 no ranking.** O valor absoluto da dívida inicial no SCR é relevante como base de risco. |
| **Pilar 3: Viabilidade e Contexto** | `num__limite_cartao` | **Top 5 no ranking.** O valor do limite é crítico para medir a exposição total ao risco. |
| **Pilar 3: Viabilidade e Contexto** | `num__taxa_inadimplencia_pf` | Contextualiza a pressão macroeconômica no período de teste (OOT). |

---

### 6. Conclusão Estratégica: Performance Comprovada e Implantação

#### 6.1. O Ciclo se Concretizou:

1. **Fundamentação Teórica** → Hipótese de estresse agudo validada pelas features de tendência
2. **Modelo Preditivo (LightGBM)** validado com performance **robusta e de alta diferenciação** (**AUC = 0.7732**) em Validação Out-of-Time (OOT)
3. **Estratégia Otimizada** → Threshold ajustado para **0.20**, resultando em **Recall de 89%** para a Classe 1 (Rotativo)
4. **Viabilidade Operacional** → O Trade-off (Precision 32%) é gerenciado transformando Falsos Positivos em **Oportunidades de Fidelização** através da intervenção consultiva

#### 6.2. Resultados de Performance (OOT - Out-of-Time):

- **AUC Score:** 0.7732
- **Recall (Classe 1):** 89% 
- **Precision (Classe 1):** 32%
- **F1-Score (Classe 1):** 47%

#### 6.3. Próximos Passos para Implantação:

1. **Integração com Sistemas** de cobrança e atendimento ao cliente
2. **Automação de Alertas** para equipe de relacionamento
3. **Dashboard de Monitoramento** contínuo da performance do modelo
4. **Retreinamento Periódico** com dados atualizados do mercado